In [176]:
# Import libraries
import math
import networkx as nx
import numpy as np
import pandas as pd
import warnings
import igraph as ig
from ast import literal_eval
import pickle as pkl
import os
import re


# Suppress specific warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Useful functions

def make_attr_dict(*args, **kwargs): 
    
    argCount = len(kwargs)
    
    if argCount > 0:
        attributes = {}
        for kwarg in kwargs:
            attributes[kwarg] = kwargs.get(kwarg, None)
        return attributes
    else:
        return None
    
# Load data
nodes_path = "../data/Montereale_Valcellina_gdf_nodes.csv"
edges_path = "../data/Montereale_Valcellina_all_lts.csv"
nodes_data = pd.read_csv(nodes_path)
edges_data = pd.read_csv(edges_path)

def create_node_attr_dict(row):
    return make_attr_dict(coord=row.geometry, lts=int(row.lts) if not pd.isna(row.lts) else -1)

def create_edge_attr_dict(row):
    return make_attr_dict(length=row.length, edge_id=row.edge_id, coord=row.geometry, lts=row.lts, intnodes=[])

def create_edge_id(row):
    return str(sorted([row["u"], row["v"]]))

def get_min_node(row):
    return min(row["u"], row["v"])

def get_max_node(row):
    return max(row["u"], row["v"])

def convert_to_string(data):
    return str(data) if isinstance(data, list) else data

def convert_attributes(graph):
    for _, attributes in graph.nodes(data=True):
        for key, value in attributes.items():
            attributes[key] = convert_to_string(value)

    for _, _, attributes in graph.edges(data=True):
        for key, value in attributes.items():
            attributes[key] = convert_to_string(value)

# Create node and edge attribute dictionaries
nodes_data["attr_dict"] = nodes_data.apply(create_node_attr_dict, axis=1)
edges_data["edge_id"] = edges_data.apply(create_edge_id, axis=1)

# Drop duplicates
edges_data = edges_data.drop_duplicates(subset=["osmid", "oneway", "edge_id", "length"], keep="first").reset_index(drop=True)
edges_data["attr_dict"] = edges_data.apply(create_edge_attr_dict, axis=1)

# Sort and rename columns
edges_data["orig"] = edges_data.apply(get_min_node, axis=1)
edges_data["dest"] = edges_data.apply(get_max_node, axis=1)
edges_data = edges_data.drop(columns=["u", "v"])

# Create graph
mnw = nx.Graph()
mnw.add_nodes_from(nodes_data[["osmid", "attr_dict"]].itertuples(index=False))
mnw.add_edges_from(edges_data[["orig", "dest", "attr_dict"]].itertuples(index=False))

# Convert attributes to string
convert_attributes(mnw)

# Save the graph
nx.write_graphml(mnw, "../data/mnw.graphml")

# Get connected components
cd_nodeset = list(nx.connected_components(mnw))
print(f"number of disconnected components on mnw: {len(cd_nodeset)}")

# Extract component details
cd_size = [len(comp) for comp in cd_nodeset]
cd_network = [nx.subgraph(mnw, comp) for comp in cd_nodeset]
cd_coord_dict = [nx.get_edge_attributes(net, "coord") for net in cd_network]
cd_coord_list = [[coords[key] for key in coords.keys()] for coords in cd_coord_dict]

# Create dataframe with component details
comps = pd.DataFrame({
    'nodeset': cd_nodeset, 
    'size': cd_size,
    'network': cd_network,
    'coord': cd_coord_list
})

# Get the largest connected component
lcc = comps["size"].max()
print(f"size of lcc: {lcc}")

comps = comps.sort_values(by="size", ascending=False).reset_index(drop=True)
mnwl_nodes = comps.iloc[0]["nodeset"]
mnwl_edges = edges_data[edges_data["orig"].isin(mnwl_nodes)].copy().reset_index(drop=True)
mnwl = nx.subgraph(mnw, mnwl_nodes)





number of disconnected components on mnw: 328
size of lcc: 901


In [ ]:
def get_sorted_neighbors(graph, node):
    return sorted([n for n in graph.neighbors(node)])

def simplify_network(mnwl):
    H = mnwl.copy()
    simplify_further = True
    run = 0

    while simplify_further:
        run += 1
        print(f"Run {run}")

        points_all_list = sorted(list(H.nodes))
        degrees_all_list = [H.degree(node) for node in points_all_list]

        pointsall = pd.DataFrame({
            "osmid": points_all_list,
            "d": degrees_all_list,
            "remove": None
        })

        pointsend = pointsall[pointsall["d"] != 2].copy()
        pointsend["remove"] = False

        pointsd2 = pointsall[pointsall["d"] == 2].copy()
        pointsd2["remove"] = True

        nodes_interstitial = pointsd2[pointsd2["remove"] == True].copy()

        eint = nodes_interstitial.copy()
        eint["orig"] = eint["osmid"].apply(lambda x: get_sorted_neighbors(H, x)[0])
        eint["dest"] = eint["osmid"].apply(lambda x: get_sorted_neighbors(H, x)[1])

        lendict = nx.get_edge_attributes(H, "length")
        eint["length_new"] = eint.apply(lambda x: np.sum([lendict[tuple(sorted(edge))] for edge in H.edges(x.osmid)]), axis=1)

        stack = list(np.unique(eint["osmid"]))

        Hprior = H.copy()

        intnodesdict = nx.get_edge_attributes(H, "intnodes")
        edgecoorddict = nx.get_edge_attributes(H, "coord")

        nodes_removed = 0

        while stack:
            mynode = stack.pop()

            for n in nx.neighbors(Hprior, mynode):
                if n in stack:
                    stack.remove(n)

            u, v = eint.loc[eint["osmid"] == mynode, ["orig", "dest"]].values[0]

            if (u, v) not in H.edges:
                myintnodes = [intnodesdict[tuple(sorted(edge))] for edge in H.edges(mynode)]
                myintnodes.append([mynode])

                H.add_edge(u, v,
                           length=eint.loc[eint["osmid"] == mynode]["length_new"].values[0],
                           intnodes=myintnodes,
                           edge_id=str(sorted([u, v])),
                           coord=edgecoorddict.get(tuple(sorted([u, mynode])), []) + edgecoorddict.get(tuple(sorted([v, mynode])), []))

                H.remove_node(mynode)
                nodes_removed += 1

        if nodes_removed == 0:
            simplify_further = False
            print("Done")

    return H

# Simplify the network and save
H = simplify_network(mnwl)
nx.write_gpickle(H, "../data/H.gpickle")

# conversion to igraph
h = ig.Graph.from_networkx(H)
# to read in again: Graph.Read_Pickle()

# eids: "conversion table" for edge ids from igraph to nx 
eids_nx = [tuple(sorted(literal_eval(h.es(i)["edge_id"][0]))) for i in range(len(h.es))]
eids_ig = [i for i in range(len(h.es))]
eids_conv = pd.DataFrame({"nx": eids_nx, "ig": eids_ig})

# nids: "conversion table" for node ids from igraph to nx
nids_nx = [h.vs(i)["_nx_name"][0] for i in range(len(h.vs))]
nids_ig = [i for i in range(len(h.vs))]
nids_conv = pd.DataFrame({"nx": nids_nx, "ig": nids_ig})

eids_conv.to_pickle("../data/eids_conv.pickle")
nids_conv.to_pickle("../data/nids_conv.pickle")

#eids_conv_read = pd.read_pickle("../data/eids_conv.pickle")
#print(eids_conv_read.head())

#nids_conv_read = pd.read_pickle("../data/nids_conv.pickle")
#print(nids_conv_read.head())

# extract edge and node attributes as dictionaries

# make data frame of ebc with:
ebc = pd.DataFrame({"edge_ig": [e.index for e in h.es]}) # igraph edge ID
ebc["edge_nx"] = ebc.apply(lambda x: tuple(literal_eval(h.es[x.edge_ig]["edge_id"])), axis = 1) # nx edge ID
ebc["length"] = ebc.apply(lambda x: h.es[x.edge_ig]["length"], axis = 1) # length in meters

# compute ebcs:
ebc["ebc_inf"] = h.edge_betweenness(directed = False, cutoff = None, weights = "length") # "standard" ebc
ebc["ebc_lambda"] = h.edge_betweenness(directed = False, cutoff = 2500, weights = "length") # ebc only including *paths* below 2500m
ebc.to_pickle("../data/ebc.pickle")

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Done


In [178]:
# Step 1-2: Identify and prioritize gaps
ced = nx.get_edge_attributes(H, "coord")

# Useful functions

# computes pathlength by nx - handling error message if nodes are not connected/not part of the network
def pathlength_if_connected(my_nw, my_o, my_d):
    try:
        return nx.dijkstra_path_length(my_nw, my_o, my_d, weight="length")
    except nx.NetworkXNoPath:
        return math.inf
    
def extract_coords_from_linestring(linestring):
    # Extract coordinates using regex
    matches = re.findall(r'(\d+\.\d+) (\d+\.\d+)', linestring)
    return [(float(y), float(x)) for x, y in matches]

def get_path_coords(my_path, my_coorddict):
    pathcoords = []
    for edge_id in my_path:
        linestring = my_coorddict[tuple(sorted(edge_id))]
        edge_coords = extract_coords_from_linestring(linestring)
        pathcoords.append(edge_coords)
    return pathcoords

# Identify all the gaps:

# shortest_path_list = list of shortest paths for all possible contact-to-contact node combinations

shortest_path_list = []

if not os.path.exists("../data/chunks"):
    os.mkdir("../data/chunks")

# ALL CONTACT NODES FROM THE NETWORK
nodestack = [node.index for node in h.vs()]

count = 0

while nodestack:
    
    node = nodestack.pop()
    
    shortest_path_list = shortest_path_list + h.get_shortest_paths(node, to=nodestack, weights="length", mode="out", output = "epath")
    
    if len(shortest_path_list) >= 2*10**5:
        with open("../data/chunks/c" + str(count) + ".pickle", 'wb') as handle:
            pkl.dump(shortest_path_list, handle, protocol=pkl.HIGHEST_PROTOCOL)
        del(shortest_path_list)
        count += 1
        shortest_path_list = []
        
        
# Saving the last chunck (WITH LEN < 2*10**5)
with open("../data/chunks/c" + str(count) + ".pickle", 'wb') as handle:
    pkl.dump(shortest_path_list, handle, protocol=pkl.HIGHEST_PROTOCOL)

del(shortest_path_list)

cs = set()
for edge in eids_conv["ig"]:
        cs.add(edge)

mygaps = []

# Chunkwise:

mychunks = ["../data/chunks/" + filename for filename in os.listdir("../data/chunks/")]

for chunk in mychunks:
    
    with open(chunk, 'rb') as f:
        pathlist = pkl.load(f)

    # adding the item to the gaplist only if it consists of only-car-edges
    gaplist = [item for item in pathlist if set(item).issubset(cs)]

    mygaps = mygaps + gaplist
    
    del(gaplist, pathlist)
    
print(len(mygaps), " gaps found")

# remove chunks (not needed anymore)
for chunk in mychunks:
    os.remove(chunk)
os.rmdir("../data/chunks")

# Convert the shortest_path list to a dataframe and add length, origin and destination

# to df
mygaps = pd.DataFrame({"path": mygaps})

# add length
mygaps["length"] = mygaps.apply(lambda x: np.sum([h.es[e]["length"] for e in x.path]), axis = 1)

# add path in nx edge id
mygaps["path_nx"] = mygaps.apply(lambda x: 
                                 [tuple(sorted(literal_eval(h.es[edge]["edge_id"]))) for edge in x.path], 
                                 axis = 1)

# add origin and destination nodes
# (separate procedure for gaps with edgenumber (enr) == 1 vs. gaps with enr > 1)
mygaps["enr"] = mygaps.apply(lambda x: len(x.path), axis = 1)
mygaps["o_nx"] = None
mygaps["d_nx"] = None
mygaps.loc[mygaps["enr"]==1, "o_nx"] = mygaps[mygaps["enr"] == 1].apply(lambda x: x.path_nx[0][0], axis = 1)
mygaps.loc[mygaps["enr"]==1, "d_nx"] = mygaps[mygaps["enr"] == 1].apply(lambda x: x.path_nx[0][1], axis = 1)
mygaps.loc[mygaps["enr"]!=1, "o_nx"] = mygaps[mygaps["enr"]!=1].apply(lambda x: set(x.path_nx[0]).difference(x.path_nx[1]).pop(), axis = 1)
mygaps.loc[mygaps["enr"]!=1, "d_nx"] = mygaps[mygaps["enr"]!=1].apply(lambda x: set(x.path_nx[-1]).difference(x.path_nx[-2]).pop(), axis = 1)
mygaps.drop(columns = "enr", inplace = True)

# add coordinates for  plotting
mygaps["gapcoord"] = mygaps.apply(lambda x: get_path_coords(x.path_nx, ced), axis = 1)

196251  gaps found


In [179]:
print(mygaps["gapcoord"].head())


0    [[(5118125.883698401, 783143.0910435098), (511...
1    [[(5118125.883698401, 783143.0910435098), (511...
2    [[(5118125.883698401, 783143.0910435098), (511...
3    [[(5118125.883698401, 783143.0910435098), (511...
4    [[(5118125.883698401, 783143.0910435098), (511...
Name: gapcoord, dtype: object
